In [17]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

In [18]:
import pandas as pd
data_types = {"author_id": "object", "id": "object"}
df_tweets = pd.read_csv("./tweets_01.csv", index_col=False, header=None, names=['author_id', 'id', 'referenced_tweets', 'text', 'date', 'cleaned_text', 'stopped_text', 'lem_text'], dtype=data_types, parse_dates=["date"])

In [19]:
df_tweets = df_tweets[df_tweets["cleaned_text"].isna()==False]

In [20]:
stopwords = frozenset(['learn', 'distance', 'remote', 'distancelearning', 'remotelearning', 'onlinelearning', 'online', 'learning'])

In [21]:
new_stopwords = stopwords.union(gensim.parsing.preprocessing.STOPWORDS)

In [22]:
df_tweets = df_tweets["cleaned_text"].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_tweets = df_tweets.apply(lambda x: " ".join(x for x in x.split() if x not in new_stopwords and len(x)>3))
text_sep = df_tweets.apply(lambda x : x.split())

In [32]:
text = df_tweets.apply(lambda x : [x for x in x.split()])
text_arr = []
text_arr.extend(text.tolist())

In [23]:
dictionary = gensim.corpora.Dictionary(text_sep)

In [24]:
dictionary.filter_extremes(no_below=10, no_above=0.5, keep_n=100000)

In [25]:
bow_corpus = [dictionary.doc2bow(doc) for doc in text_sep]

In [96]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=6, id2word=dictionary, passes=10)

In [58]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.016*"today" + 0.012*"video" + 0.011*"live" + 0.010*"morning" + 0.009*"zoom" + 0.009*"google" + 0.009*"students" + 0.009*"check" + 0.008*"class" + 0.008*"week"
Topic: 1 
Words: 0.018*"snow" + 0.012*"days" + 0.009*"class" + 0.007*"game" + 0.007*"president" + 0.007*"play" + 0.007*"trump" + 0.006*"house" + 0.005*"games" + 0.005*"history"
Topic: 2 
Words: 0.035*"february" + 0.021*"weather" + 0.018*"higher" + 0.017*"productivity" + 0.016*"npccertificate" + 0.015*"npcindia" + 0.014*"npcindiagov" + 0.013*"safety" + 0.013*"wateraudit" + 0.013*"hrcommunity"
Topic: 3 
Words: 0.031*"students" + 0.019*"great" + 0.015*"week" + 0.014*"teachers" + 0.014*"work" + 0.013*"thank" + 0.012*"year" + 0.009*"teaching" + 0.008*"today" + 0.007*"thanks"
Topic: 4 
Words: 0.028*"kids" + 0.026*"education" + 0.021*"math" + 0.018*"problem" + 0.017*"energyefficiency" + 0.017*"chance" + 0.016*"works" + 0.016*"homeschooling" + 0.015*"times" + 0.014*"safety"
Topic: 5 
Words: 0.052*"schools" + 0.047*"sch

In [56]:
cm = CoherenceModel(model=lda_model, corpus=bow_corpus, dictionary=dictionary, coherence='u_mass')
print(cm.get_coherence())

-4.734975405607225


In [53]:
cv =  CoherenceModel(model=lda_model, texts=text_arr, dictionary=dictionary, coherence='c_v')
print(cv.get_coherence())

0.41680321949769183


In [42]:
import pandas as pd
import re
data_types = {"author_id": "object", "id": "object"}
df = pd.read_csv("./tweets_01.csv", index_col=False, header=None, names=['author_id', 'id', 'referenced_tweets', 'text', 'date', 'cleaned_text', 'stopped_text', 'lem_text'], dtype=data_types, parse_dates=["date"])

def clean_text(text):    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions -V
    text = re.sub('#', '', text) # Removing '#' hash tag
    text = re.sub('RT', '', text) # Removing RT -V
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink -V
    text = emoji_pattern.sub(r'', text)  # remove emojis -V
    text = re.sub('_', '', text)
    return text

df["cleaned_text"] = df["text"].apply(clean_text)
data = df.cleaned_text.values.tolist()

In [59]:
def format_topics_sentences(ldamodel=lda_model, corpus=bow_corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=bow_corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

Document_No  Dominant_Topic  Topic_Perc_Contrib  \
0            0             6.0              0.4322   
1            1             3.0              0.3786   
2            2             0.0              0.7213   
3            3             8.0              0.5833   
4            4             9.0              0.5798   
5            5             3.0              0.7405   
6            6             7.0              0.4741   
7            7             7.0              0.8144   
8            8             0.0              0.4846   
9            9             0.0              0.8873   

                                            Keywords  \
0  students, education, help, access, pandemic, s...   
1  students, great, week, teachers, work, thank, ...   
2  today, video, live, morning, zoom, google, stu...   
3  education, edtech, edchat, health, mental, tea...   
4  kids, school, dont, teachers, like, time, home...   
5  students, great, week, teachers, work, thank, ...   
6  free, join, webinar, course, register, virtual...   
7  free, join, webinar, course, register, virtual...   
8  today, video, live, morning, zoom, google, stu...   
9  today, video, live, morning, zoom, google, stu...   

                                                Text  
0  BTS fans have been changing the world one camp...  
1  “Recognizing effort and offering praise for sm...  
2  We'll be giving away one HUE Doc Camera to a t...  
3  Students are using ai to navigate onlinelearni...  
4  witten  Isn’t it 25/100k for reopening? Looks ...  
5  How fun is this!? Charlie gives some great ide...  
6  Please check your email for an important updat...  
7  Missed our webinar sessions? Don’t worry! All ...  
8  Wait...? This was a conversation  today in my ...  
9  on behalf of every student in SA, we forgot ho...

In [95]:
with pd.option_context('display.max_colwidth', None):
    print(df_dominant_topic[df_dominant_topic.Dominant_Topic == 0.0].sort_values(by='Topic_Perc_Contrib', ascending=False)["Text"][0:30])


72020                                                             Introducing... Freshers Fortnight!  Join us 28 Sept - 10 Oct for over 40 amazing sessions, exciting competitions and prize draws, and most importantly opportunities to get to know fellow OUstudents and feel part of the OU family   
95842                                                                                                                                                                                                                                 Online learning doesn't make any sense at all especially in Lebanon
78071         A bad take by a self-professed neo-nazi.\n\n1) screen time data refers to media (i.e., YouTube, App games) and NOT face-to-face ed. content\n\n2) anecdotes and select parent‘s feelings are not data\n\n3) educators are aware of the challenges to children and plan breaks and movement 
58045             LessonOfTheDay - Ndebele Numbers\n\nisiNdebele is an African language belonging to the N